# HW1

## Overview

Preparing the data, computing basic statistics and constructing simple models are essential steps for data science practice. In this homework, you will use clinical data as raw input to perform **Heart Failure Prediction**. For this homework, **Python** programming will be required. See the attached skeleton code as a start-point for the programming questions.

This homework assumes familiarity with Pandas. If you need a Pandas crash course, we recommend working through [100 Pandas Puzzles](https://github.com/ajcr/100-pandas-puzzles), the solutions are also available at that link. 

---

In [1]:
import os
import sys

DATA_PATH = "./data/"
TRAIN_DATA_PATH = DATA_PATH + "train/"
VAL_DATA_PATH = DATA_PATH + "val/"
    
sys.path.append("./")

---

## About Raw Data

For this homework, we will be using a clinical dataset synthesized from [MIMIC-III](https://www.nature.com/articles/sdata201635).

Navigate to `TRAIN_DATA_PATH`. There are three CSV files which will be the input data in this homework. 

In [2]:
!ls $TRAIN_DATA_PATH

'ls' is not recognized as an internal or external command,
operable program or batch file.


**events.csv**

The data provided in *events.csv* are event sequences. Each line of this file consists of a tuple with the format *(pid, event_id, vid, value)*. 

For example, 

```
33,DIAG_244,0,1
33,DIAG_414,0,1
33,DIAG_427,0,1
33,LAB_50971,0,1
33,LAB_50931,0,1
33,LAB_50812,1,1
33,DIAG_425,1,1
33,DIAG_427,1,1
33,DRUG_0,1,1
33,DRUG_3,1,1
```

- **pid**: De-identified patient identier. For example, the patient in the example above has pid 33. 
- **event_id**: Clinical event identifier. For example, DIAG_244 means the patient was diagnosed of disease with ICD9 code [244](http://www.icd9data.com/2013/Volume1/240-279/240-246/244/244.htm); LAB_50971 means that the laboratory test with code 50971 was conducted on the patient; and DRUG_0 means that a drug with code 0 was prescribed to the patient. Corresponding lab (drug) names can be found in `{DATA_PATH}/lab_list.txt` (`{DATA_PATH}/drug_list.txt`).
- **vid**: Visit identifier. For example, the patient has two visits in total. Note that vid is ordinal. That is, visits with bigger vid occour after that with smaller vid.
- **value**: Contains the value associated to an event (always 1 in the synthesized dataset).

**hf_events.csv**

The data provided in *hf_events.csv* contains pid of patients who have been diagnosed with heart failure (i.e., DIAG_398, DIAG_402, DIAG_404, DIAG_428) in at least one visit. They are in the form of a tuple with the format *(pid, vid, label)*. For example,

```
156,0,1
181,1,1
```

The vid indicates the index of the first visit with heart failure of that patient and a label of 1 indicates the presence of heart failure. **Note that only patients with heart failure are included in this file. Patients who are not mentioned in this file have never been diagnosed with heart failure.**

**event_feature_map.csv**

The *event_feature_map.csv* is a map from an event_id to an integer index. This file contains *(idx, event_id)* pairs for all event ids.

## 1 Descriptive Statistics [20 points]

Before starting analytic modeling, it is a good practice to get descriptive statistics of the input raw data. In this question, you need to write code that computes various metrics on the data described previously. A skeleton code is provided to you as a starting point.

The definition of terms used in the result table are described below:

- **Event count**: Number of events recorded for a given patient.
- **Encounter count**: Number of visits recorded for a given patient.

Note that every line in the input file is an event, while each visit consists of multiple events.

**Complete the following code cell to implement the required statistics.**

Please be aware that **you are NOT allowed to change the filename and any existing function declarations.** Only `numpy`, `scipy`, `scikit-learn`, `pandas` and other built-in modules of python will be available for you to use. The use of `pandas` library is suggested. 

In [3]:
import time
import pandas as pd
import numpy as np
import datetime

# PLEASE USE THE GIVEN FUNCTION NAME, DO NOT CHANGE IT.

def read_csv(filepath=TRAIN_DATA_PATH):

    '''
    Read the events.csv and hf_events.csv files. 
    Variables returned from this function are passed as input to the metric functions.
    
    NOTE: remember to use `filepath` whose default value is `TRAIN_DATA_PATH`.
    '''
    
    events = pd.read_csv(filepath + 'events.csv')
    hf = pd.read_csv(filepath + 'hf_events.csv')

    return events, hf

def event_count_metrics(events, hf):

    '''
    TODO : Implement this function to return the event count metrics.
    
    Event count is defined as the number of events recorded for a given patient.
    '''

    # your code here
    hf_pids = hf.loc[:, 'pid']
    num_ev_by_patient = (events.groupby(['pid']).sum())
    ev_by_norm_patient = num_ev_by_patient.loc[~num_ev_by_patient.index.isin(hf_pids), 'value']
    ev_by_hf_patient = num_ev_by_patient.loc[num_ev_by_patient.index.isin(hf_pids), 'value']

    avg_hf_event_count = ev_by_hf_patient.mean()
    max_hf_event_count = ev_by_hf_patient.max()
    min_hf_event_count = ev_by_hf_patient.min()
    avg_norm_event_count = ev_by_norm_patient.mean()
    max_norm_event_count = ev_by_norm_patient.max()
    min_norm_event_count = ev_by_norm_patient.min()

    return avg_hf_event_count, max_hf_event_count, min_hf_event_count, \
           avg_norm_event_count, max_norm_event_count, min_norm_event_count

def encounter_count_metrics(events, hf):

    '''
    TODO : Implement this function to return the encounter count metrics.
    
    Encounter count is defined as the number of visits recorded for a given patient. 
    '''
    
    avg_hf_encounter_count = None
    max_hf_encounter_count = None
    min_hf_encounter_count = None
    avg_norm_encounter_count = None
    max_norm_encounter_count = None
    min_norm_encounter_count = None
    
    # your code here
    hf_pids = hf.loc[:, 'pid']
    num_vis_by_patient = (events.groupby(['pid'])[['vid']].nunique())
    num_vis_by_norm_patient = num_vis_by_patient.loc[~num_vis_by_patient.index.isin(hf_pids), 'vid']
    num_vis_by_hf_patient = num_vis_by_patient.loc[num_vis_by_patient.index.isin(hf_pids), 'vid']

    avg_hf_encounter_count = num_vis_by_hf_patient.mean()
    max_hf_encounter_count = num_vis_by_hf_patient.max()
    min_hf_encounter_count = num_vis_by_hf_patient.min()
    avg_norm_encounter_count = num_vis_by_norm_patient.mean()
    max_norm_encounter_count = num_vis_by_norm_patient.max()
    min_norm_encounter_count = num_vis_by_norm_patient.min()

    return avg_hf_encounter_count, max_hf_encounter_count, min_hf_encounter_count, \
           avg_norm_encounter_count, max_norm_encounter_count, min_norm_encounter_count

In [4]:
events, hf = read_csv(TRAIN_DATA_PATH)
#print(events.shape[0])
#print(events.head())

hf_pids = hf.loc[:, 'pid']
print(hf_pids)
ev_by_patient = (events.groupby(['pid']).sum())
print("ev_by_patient=\n", ev_by_patient)

ev_by_norm_patient = ev_by_patient.loc[~ev_by_patient.index.isin(hf_pids), 'value']
#print("ev_by_norm_patient=\n", ev_by_norm_patient)

#print(ev_by_norm_patient.mean())
#print(ev_by_norm_patient.max())
#print(ev_by_norm_patient.min())

ev_by_hf_patient = ev_by_patient.loc[ev_by_patient.index.isin(hf_pids), 'value']

#print(ev_by_hf_patient.mean())
#print(ev_by_hf_patient.max())
#print(ev_by_hf_patient.min())

num_vis_by_patient = (events.groupby(['pid'])[['vid']].nunique())
print("num_vis_by_patient=\n", num_vis_by_patient)

num_vis_by_norm_patient = num_vis_by_patient.loc[~num_vis_by_patient.index.isin(hf_pids), 'vid']
print(num_vis_by_norm_patient)

print(num_vis_by_norm_patient.mean())
print(num_vis_by_norm_patient.max())
print(num_vis_by_norm_patient.min())

num_vis_by_hf_patient = num_vis_by_patient.loc[num_vis_by_patient.index.isin(hf_pids), 'vid']
#print(num_vis_by_hf_patient)

print(num_vis_by_hf_patient.mean())
print(num_vis_by_hf_patient.max())
print(num_vis_by_hf_patient.min())


0          33
1          64
2         156
3         181
4         199
        ...  
2955    99784
2956    99788
2957    99825
2958    99841
2959    99971
Name: pid, Length: 2960, dtype: int64
ev_by_patient=
        vid  value
pid              
33      75    122
64      57    132
78      81    209
156     70    187
181     64    165
...    ...    ...
99788   54    107
99800   79    161
99825  450    238
99841  215    192
99971   71    143

[4000 rows x 2 columns]
num_vis_by_patient=
        vid
pid       
33       2
64       2
78       2
156      2
181      2
...    ...
99788    2
99800    2
99825    4
99841    3
99971    2

[4000 rows x 1 columns]
pid
78       2
190      2
333      2
695      2
701      2
        ..
99102    2
99381    2
99586    2
99729    2
99800    2
Name: vid, Length: 1040, dtype: int64
2.189423076923077
11
1
2.8060810810810812
34
2


In [5]:
df = pd.DataFrame({'pid': [1, 1, 2, 2, 2, 3], 'vid': [0, 1, 0, 1, 5, 0]})
x = df.groupby(['pid']).nunique()
print(x)

     vid
pid     
1      2
2      3
3      1


In [6]:
'''
DO NOT MODIFY THIS.
'''

events, hf = read_csv(TRAIN_DATA_PATH)
print(events.head())

#Compute the event count metrics
start_time = time.time()
event_count = event_count_metrics(events, hf)
end_time = time.time()
print(("Time to compute event count metrics: " + str(end_time - start_time) + "s"))
print(event_count)

#Compute the encounter count metrics
start_time = time.time()
encounter_count = encounter_count_metrics(events, hf)
end_time = time.time()
print(("Time to compute encounter count metrics: " + str(end_time - start_time) + "s"))
print(encounter_count)

   pid  event_id  vid  value
0   33  DIAG_244    0      1
1   33  DIAG_414    0      1
2   33  DIAG_427    0      1
3   33  DIAG_585    0      1
4   33  DIAG_V45    0      1
Time to compute event count metrics: 0.2897145748138428s
(188.9375, 2046, 28, 118.64423076923077, 1014, 6)
Time to compute encounter count metrics: 0.4781181812286377s
(2.8060810810810812, 34, 2, 2.189423076923077, 11, 1)


In [7]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

events, hf = read_csv(TRAIN_DATA_PATH)
event_count = event_count_metrics(events, hf)
assert event_count == (188.9375, 2046, 28, 118.64423076923077, 1014, 6), "event_count failed!"



In [8]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

events, hf = read_csv(TRAIN_DATA_PATH)
encounter_count = encounter_count_metrics(events, hf)
assert encounter_count == (2.8060810810810812, 34, 2, 2.189423076923077, 11, 1), "encounter_count failed!"



## 2 Feature construction [40 points] 

It is a common practice to convert raw data into a standard data format before running real machine learning models. In this question, you will implement the necessary python functions in this script. You will work with *events.csv*, *hf_events.csv* and *event_feature_map.csv* files provided in **TRAIN_DATA_PATH** folder. The use of `pandas` library in this question is recommended. 

Listed below are a few concepts you need to know before beginning feature construction (for details please refer to lectures). 

<img src="img/window.jpg" width="600"/>

- **Index vid**: Index vid is evaluated as follows:
  - For heart failure patients: Index vid is the vid of the first visit with heart failure for that patient (i.e., vid field in *hf_events.csv*). 
  - For normal patients: Index vid is the vid of the last visit for that patient (i.e., vid field in *events.csv*). 
- **Observation Window**: The time interval you will use to identify relevant events. Only events present in this window should be included while constructing feature vectors.
- **Prediction Window**: A fixed time interval that is to be used to make the prediction.

In the example above, the index vid is 3. Visits with vid 0, 1, 2 are within the observation window. The prediction window is between visit 2 and 3.

### 2.1 Compute the index vid [10 points]

Use the definition provided above to compute the index vid for all patients. Complete the method `read_csv` and `calculate_index_vid` provided in the following code cell. 

In [9]:
import pandas as pd
import datetime


def read_csv(filepath=TRAIN_DATA_PATH):
    
    '''
    Read the events.csv, hf_events.csv and event_feature_map.csv files.
    
    NOTE: remember to use `filepath` whose default value is `TRAIN_DATA_PATH`.
    '''

    events = pd.read_csv(filepath + 'events.csv')
    hf = pd.read_csv(filepath + 'hf_events.csv')
    feature_map = pd.read_csv(filepath + 'event_feature_map.csv')

    return events, hf, feature_map


def calculate_index_vid(events, hf):
    
    '''
    TODO: This function needs to be completed.

    Suggested steps:
        1. Create list of normal patients (hf_events.csv only contains information about heart failure patients).
        2. Split events into two groups based on whether the patient has heart failure or not.
        3. Calculate index vid for each patient.
    
    IMPORTANT:
        `indx_vid` should be a pd dataframe with header `['pid', 'indx_vid']`.
    '''

    indx_vid = ''
    
    # your code here
    hf_pids = hf.loc[:, ['pid']]
    ev_by_norm_patient = events.loc[~events['pid'].isin(hf_pids['pid']), :]
    ivid_norm_patient = ev_by_norm_patient.groupby(['pid'])[['vid']].max()
    ivid_norm_patient.reset_index(drop=False, inplace=True)

    ev_by_hf_patient = pd.merge(events, hf,  how='inner',
                          left_on=['pid','vid'], right_on = ['pid','vid'])

    ivid_hf_patient = ev_by_hf_patient.drop_duplicates(subset=['pid', 'vid'])
    ivid_hf_patient = ivid_hf_patient.loc[: , ['pid', 'vid']]
    ivid_hf_patient.reset_index(drop=True, inplace=True)

    indx_vid = pd.concat([ivid_norm_patient, ivid_hf_patient])
    indx_vid.reset_index(drop=True, inplace=True)
    indx_vid.rename({'vid': 'indx_vid'}, axis=1, inplace=True)
    
    return indx_vid

In [10]:
print(hf.shape)

hf_pids = hf.loc[:, ['pid']]
#print(hf_pids)

#print(~events['pid'].isin(hf_pids['pid']))

ev_by_norm_patient = events.loc[~events['pid'].isin(hf_pids['pid']), :]
#print(ev_by_norm_patient)
ivid_norm_patient = ev_by_norm_patient.groupby(['pid'])[['vid']].max()
#### Note: This seems ok
#print(ivid_norm_patient)
ivid_norm_patient.reset_index(drop=False, inplace=True)
print(ivid_norm_patient)
#print(ivid_norm_patient[ivid_norm_patient.index == 1230])
#print(ivid_norm_patient[ivid_norm_patient.index == 78)
#print(ivid_norm_patient.loc[ivid_norm_patient.loc[:, 'vid'] > 1, : ])

#ev_by_hf_patient = events.loc[events['pid'].isin(hf_pids['pid']), ['pid', 'vid']]
#print(ev_by_hf_patient.loc[ev_by_hf_patient.loc[:,'pid'] == 181, :])
ev_by_hf_patient = pd.merge(events, hf,  how='inner',
                      left_on=['pid','vid'], right_on = ['pid','vid'])

print(ev_by_hf_patient.shape)
#print(ev_by_hf_patient[ev_by_hf_patient.pid == 33])
#ivid_hf_patient = ev_by_hf_patient.groupby(['pid'])[['vid']].nunique()
ivid_hf_patient = ev_by_hf_patient.drop_duplicates(subset=['pid', 'vid'])
ivid_hf_patient = ivid_hf_patient.loc[: , ['pid', 'vid']]
ivid_hf_patient.reset_index(drop=True, inplace=True)
print(ivid_hf_patient)

indx_vid = pd.concat([ivid_norm_patient, ivid_hf_patient])
#indx_vid['pid'] = indx_vid.index
indx_vid.reset_index(drop=True, inplace=True)
indx_vid.rename({'vid': 'indx_vid'}, axis=1, inplace=True)
print(indx_vid)
print(indx_vid[indx_vid.pid == 33])
#print(x.loc[x.loc[:,'pid'] == 33, :].loc[x.loc[:,'vid'] == 0, :])

(2960, 3)
        pid  vid
0        78    1
1       190    1
2       333    1
3       695    1
4       701    1
...     ...  ...
1035  99102    1
1036  99381    1
1037  99586    1
1038  99729    1
1039  99800    1

[1040 rows x 2 columns]
(201290, 5)
        pid  vid
0        33    0
1        64    0
2       156    0
3       181    1
4       199    1
...     ...  ...
2955  99784    2
2956  99788    1
2957  99825    1
2958  99841    0
2959  99971    1

[2960 rows x 2 columns]
        pid  indx_vid
0        78         1
1       190         1
2       333         1
3       695         1
4       701         1
...     ...       ...
3995  99784         2
3996  99788         1
3997  99825         1
3998  99841         0
3999  99971         1

[4000 rows x 2 columns]
      pid  indx_vid
1040   33         0


In [11]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

events, hf, feature_map = read_csv(TRAIN_DATA_PATH)
indx_vid_df = calculate_index_vid(events, hf)
assert indx_vid_df.shape == (4000, 2), "calculate_index_vid failed!"

indx_vid = dict(list(zip(indx_vid_df.pid, indx_vid_df.indx_vid)))
assert indx_vid[78] == 1, "calculate_index_vid failed!"
assert indx_vid[1230] == 5, "calculate_index_vid failed!"




### 2.2 Filter events [10 points]

Remove the events that occur outside the observation window. That is, all events in visits before index vid. Complete the method *filter_events* provided in the following code cell.

In [12]:
def filter_events(events, indx_vid):
    
    '''
    TODO: This function needs to be completed.

    Suggested steps:
        1. Join indx_vid with events on pid.
        2. Filter events occuring in the observation window [:, index vid) (Note that the right side is OPEN).
    
    
    IMPORTANT:
        `filtered_events` should be a pd dataframe withe header  `['pid', 'event_id', 'value']`.
    '''

    filtered_events = None
    
    # your code here
    joined = pd.merge(events, indx_vid, on="pid")

    fev = joined.loc[(joined['vid'] < joined['indx_vid']), ['pid', 'event_id', 'value']]
    filtered_events = fev.reset_index(drop=True)
    
    return filtered_events

In [13]:
print(events.shape)
print(indx_vid_df.shape)

joined = pd.merge(events, indx_vid_df, on="pid")
print(joined)
x=joined

#print(x[(x['vid'] < x['indx_vid']) & (x['pid'] == 78)])

fev = x.loc[(x['vid'] < x['indx_vid']), ['pid', 'event_id', 'indx_vid']]
hf0 = x.loc[(x['indx_vid'] == 0), ['pid', 'event_id', 'indx_vid']]
fev = fev.reset_index(drop=True)
print(fev)
print(hf0.shape)
print(fev[fev.pid == 33].shape)
print(hf0[hf0.pid == 78].shape)
print(fev[fev.pid.isin(hf0['pid'])].shape)

#print(x.isnull().values.any())
#print(events.loc[events.loc[:,'pid'] == 33, :])

(682645, 4)
(4000, 2)
          pid  event_id  vid  value  indx_vid
0          33  DIAG_244    0      1         0
1          33  DIAG_414    0      1         0
2          33  DIAG_427    0      1         0
3          33  DIAG_585    0      1         0
4          33  DIAG_V45    0      1         0
...       ...       ...  ...    ...       ...
682640  99971   DRUG_49    1      1         1
682641  99971   DRUG_71    1      1         1
682642  99971  DRUG_423    1      1         1
682643  99971  DRUG_136    1      1         1
682644  99971  DRUG_202    1      1         1

[682645 rows x 5 columns]
          pid  event_id  indx_vid
0          78  DIAG_278         1
1          78  DIAG_285         1
2          78  DIAG_300         1
3          78  DIAG_311         1
4          78  DIAG_338         1
...       ...       ...       ...
169099  99971    DRUG_1         1
169100  99971   DRUG_50         1
169101  99971  DRUG_111         1
169102  99971   DRUG_29         1
169103  99971  DRUG_210  

In [13]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

events, hf, feature_map = read_csv(TRAIN_DATA_PATH)
indx_vid = calculate_index_vid(events, hf)
filtered_events = filter_events(events, indx_vid)
print(filtered_events.shape)
assert filtered_events[filtered_events.pid == 78].shape == (128, 3), "filter_events failed!"




(169104, 3)


### 2.3 Aggregate events [10 points]

To create features suitable for machine learning, we will need to aggregate the events for each patient as follows:

- **count** occurences for each event.

Each event type will become a feature and we will directly use event_id as feature name. For example, given below raw event sequence for a patient,

```
33,DIAG_244,0,1
33,LAB_50971,0,1
33,LAB_50931,0,1
33,LAB_50931,0,1
33,DIAG_244,1,1
33,DIAG_427,1,1
33,DRUG_0,1,1
33,DRUG_3,1,1
33,DRUG_3,1,1
```

We can get feature value pairs *(event_id, value)* for this patient with ID *33* as
```
(DIAG_244, 2.0)
(LAB_50971, 1.0)
(LAB_50931, 2.0)
(DIAG_427, 1.0)
(DRUG_0, 1.0)
(DRUG_3, 2.0)
```

Next, replace each *event_id* with the *feature_id* provided in *event_feature_map.csv*.

```
(146, 2.0)
(1434, 1.0)
(1429, 2.0)
(304, 1.0)
(898, 1.0)
(1119, 2.0)
```

Lastly, in machine learning algorithm like logistic regression, it is important to normalize different features into the same scale. We will use the [min-max normalization](http://stats.stackexchange.com/questions/70801/how-to-normalize-data-to-0-1-range) approach. (Note: we define $min(x)$ is always 0, i.e. the scale equation become $x$/$max(x)$).

Complete the method *aggregate_events* provided in the following code cell.

In [14]:
def aggregate_events(filtered_events_df, hf_df, feature_map_df):
    
    '''
    TODO: This function needs to be completed.

    Suggested steps:
        1. Replace event_id's with index available in event_feature_map.csv.
        2. Aggregate events using count to calculate feature value.
        3. Normalize the values obtained above using min-max normalization(the min value will be 0 in all scenarios).
    
    
    IMPORTANT:
        `aggregated_events` should be a pd dataframe with header `['pid', 'feature_id', 'feature_value']`.
    '''
    
    aggregated_events = None
    
    # your code here
    agg_events = filtered_events_df.groupby(['pid', 'event_id']).sum()
    agg_events.reset_index(drop=False, inplace=True)

    max_events = agg_events.loc[:, ['event_id', 'value']]
    max_events = max_events.groupby(['event_id']).max()
    max_events.reset_index(drop=False, inplace=True)
    max_events = pd.merge(max_events, feature_map_df, on="event_id")
    max_events.rename({'value': 'max', 'idx': 'feature_id'}, axis=1, inplace=True)

    joined_efm = pd.merge(agg_events, max_events, on="event_id")
    joined_efm['feature_value'] = joined_efm['value'] / joined_efm['max']

    aggregated_events = joined_efm.loc[:, ['pid', 'feature_id', 'feature_value']]

    return aggregated_events

In [15]:
events_in, hf_in, feature_map_in = read_csv(TRAIN_DATA_PATH)
events_in = events_in.loc[:1000]
hf_in = hf_in.loc[:100]

index_vid = calculate_index_vid(events_in, hf_in)
filtered_events = filter_events(events_in, index_vid)
aggregated_events = aggregate_events(filtered_events, hf, feature_map_in)
print(aggregated_events)

     pid  feature_id  feature_value
0     78          20            1.0
1     78         164            1.0
2     78         175            1.0
3    181         175            1.0
4    190         175            1.0
..   ...         ...            ...
275  190        1419            1.0
276  190        1425            1.0
277  190        1434            1.0
278  190        1445            1.0
279  190        1446            1.0

[280 rows x 3 columns]


In [16]:
events, hf, feature_map = read_csv(TRAIN_DATA_PATH)
index_vid = calculate_index_vid(events, hf)
filtered_events = filter_events(events, index_vid)

#print(feature_map.head())
#print(events[events.pid == 33])
#print(filtered_events[filtered_events.pid == 33])
print(filtered_events[(filtered_events.pid == 1230) & (filtered_events.event_id == 'DIAG_E879')])
#print(filtered_events.drop_duplicates(subset=['pid', 'event_id']))
# 14350 are the number of events with more than one occurrence per patient
print(filtered_events.shape[0] - 154754)

#joined_efm = pd.merge(filtered_events, feature_map, on="event_id")
#print(joined_efm)
#joined_efm['event_id'] = joined_efm['idx']
agg_events = filtered_events.groupby(['pid', 'event_id']).sum()
print(agg_events[agg_events.value == 7])

#print(joined_efm[joined_efm.idx > 1])
agg_events.reset_index(drop=False, inplace=True)
print(agg_events[(agg_events.event_id == 'LAB_51519') & (agg_events.value > 1)])
print(agg_events)


print(agg_events[(agg_events.pid == 84188) & (agg_events.event_id == 'LAB_51519')])
max_events = agg_events.loc[:, ['event_id', 'value']]
max_events = max_events.groupby(['event_id']).max()
max_events.reset_index(drop=False, inplace=True)
max_events = pd.merge(max_events, feature_map, on="event_id")
max_events.rename({'value': 'max', 'idx': 'feature_id'}, axis=1, inplace=True)
print(max_events)

joined_efm = pd.merge(agg_events, max_events, on="event_id")
joined_efm['feature_value'] = joined_efm['value'] / joined_efm['max']
print(joined_efm)
print(joined_efm[(joined_efm.pid == 84188) & (joined_efm.event_id == 'LAB_51519')])
print(joined_efm[(joined_efm.pid == 90401) & (joined_efm.event_id == 'LAB_51519')])

aggregated_events = joined_efm.loc[:, ['pid', 'feature_id', 'feature_value']]
print(aggregated_events)

print(aggregated_events[aggregated_events.pid == 88037])
print(aggregated_events[(aggregated_events.pid == 156)])

       pid   event_id  value
1513  1230  DIAG_E879      1
1595  1230  DIAG_E879      1
1705  1230  DIAG_E879      1
1786  1230  DIAG_E879      1
14350
                 value
pid   event_id        
72589 DIAG_070       7
      DRUG_0         7
      DRUG_186       7
      DRUG_5         7
      LAB_51301      7
84188 DIAG_038       7
      DIAG_041       7
      DIAG_456       7
      DIAG_584       7
      DIAG_E879      7
      DRUG_0         7
      DRUG_1         7
      DRUG_13        7
      DRUG_6         7
      DRUG_7         7
      DRUG_73        7
      LAB_51486      7
      LAB_51491      7
      LAB_51519      7
          pid   event_id  value
3384     2823  LAB_51519      3
18482   11147  LAB_51519      2
27559   16662  LAB_51519      2
30180   18455  LAB_51519      2
31143   19173  LAB_51519      2
35830   21950  LAB_51519      2
35954   21963  LAB_51519      2
44168   27743  LAB_51519      2
46971   29360  LAB_51519      2
59796   37511  LAB_51519      3
65966   41901 

In [17]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

events, hf, feature_map = read_csv(TRAIN_DATA_PATH)
index_vid = calculate_index_vid(events, hf)
filtered_events = filter_events(events, index_vid)
aggregated_events = aggregate_events(filtered_events, hf, feature_map)
print(aggregated_events[aggregated_events.pid == 78])
assert aggregated_events[aggregated_events.pid == 88037].shape == (29, 3), "aggregate_events failed!"



       pid  feature_id  feature_value
0       78          20       0.142857
255     78         164       1.000000
261     78         175       0.250000
432     78         182       0.125000
1140    78         190       0.500000
...    ...         ...            ...
53620   78        1465       0.250000
54085   78        1467       0.166667
54454   78        1468       0.333333
54939   78        1469       0.200000
55416   78        1470       0.250000

[127 rows x 3 columns]


### 2.4 Save in  SVMLight format [10 points]

If the dimensionality of a feature vector is large but the feature vector is sparse (i.e. it has only a few nonzero elements), sparse representation should be employed. In this problem you will use the provided data for each patient to construct a feature vector and represent the feature vector in SVMLight format.

```
<line> .=. <target> <feature>:<value> <feature>:<value>
<target> .=. 1 | 0
<feature> .=. <integer>
<value> .=. <float>
```

The target value and each of the feature/value pairs are separated by a space character. Feature/value pairs MUST be ordered by increasing feature number. **(Please do this in `save_svmlight()`.)** Features with value zero can be skipped. For example, the feature vector in SVMLight format will look like: 

```
1 2:0.5 3:0.12 10:0.9 2000:0.3
0 4:1.0 78:0.6 1009:0.2
1 33:0.1 34:0.98 1000:0.8 3300:0.2
1 34:0.1 389:0.32
```

where, 1 or 0 will indicate whether the patient has heart failure or not (i.e. the label) and it will be followed by a series of feature-value pairs **sorted** by the feature index (idx) value.

You may find *utils.py* useful. You can review the code by running `%load utils.py`.

In [18]:
%load   ./utils.py

In [41]:
import utils
import collections

def create_features(events_in, hf_in, feature_map_in):

    indx_vid = calculate_index_vid(events_in, hf_in)

    #Filter events in the observation window
    filtered_events = filter_events(events_in, indx_vid)

    #Aggregate the event values for each patient 
    aggregated_events = aggregate_events(filtered_events, hf_in, feature_map_in)

    '''
    TODO: Complete the code below by creating two dictionaries.
        1. patient_features : Key is pid and value is array of tuples(feature_id, feature_value). 
                              Note that pid should be integer.
        2. hf : Key is pid and value is heart failure label.
    '''
    patient_features = None
    hf = None
    
    # your code here
    aggregated_events['tuples'] = list(zip(aggregated_events.feature_id,
                                           aggregated_events.feature_value))

    tuppled = aggregated_events.groupby('pid')[['tuples']].aggregate(lambda f: list(f))
    patient_features = dict(zip(tuppled.index, tuppled.tuples))

    #unique_pids = aggregated_events.drop_duplicates(subset=['pid'])
    #merged = pd.merge(unique_pids, hf_in, how='outer', on='pid')
    #print(merged)
    #hf_list = list(merged.pid.isin(hf_in.pid) * 1)
    #print(sum(hf_list))

    hf = dict(zip(hf_in.pid, [1] * hf_in.shape[0]))

    return patient_features, hf

def save_svmlight(patient_features, hf, op_file):
    
    '''
    TODO: This function needs to be completed.

    Create op_file: - which saves the features in svmlight format. (See instructions in section 2.4 for detailed explanatiom)
    
    Note: Please make sure the features are ordered in ascending order, and patients are stored in ascending order as well.     
    To save the files, you could write:
        deliverable.write(bytes(f"{label} {feature_value} \n", 'utf-8'))
    '''

    deliverable = open(op_file, 'wb')
    # your code here
    for pid, features in patient_features.items():
        target = hf.get(pid, 0)
        sorted_features = sorted(features, key=lambda v: v[0])
        feature_values = utils.bag_to_svmlight(sorted_features)
        deliverable.write(bytes(f"{target} {feature_values} \n", 'utf-8'))

    deliverable.close()


events_in, hf_in, feature_map_in = read_csv(TRAIN_DATA_PATH)
patient_features, hf = create_features(events_in, hf_in, feature_map_in)
save_svmlight(patient_features, hf, 'features_svmlight.train')

events_in, hf_in, feature_map_in = read_csv(VAL_DATA_PATH)
patient_features, hf = create_features(events_in, hf_in, feature_map_in)
save_svmlight(patient_features, hf, 'features_svmlight.val')
print('done')


done


In [42]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

events_in, hf_in, feature_map_in = read_csv(TRAIN_DATA_PATH)
events_in = events_in.loc[:1000]
hf_in = hf_in.loc[:100]
patient_features, hf = create_features(events_in, hf_in, feature_map_in)
print(len(hf))
assert 78 in patient_features, "create_features is missing patients"
assert len(patient_features[78]) == 127, "create_features is wrong"
assert patient_features[78][:5] == [(20, 1.0), (164, 1.0), (175, 1.0), (182, 1.0), (190, 1.0)], "create_features is wrong"
assert len(hf) == 101, "create_features is wrong"




101


The whole pipeline:

In [249]:
def main():
    events_in, hf_in, feature_map_in = read_csv(TRAIN_DATA_PATH)
    patient_features, hf = create_features(events_in, hf_in, feature_map_in)
    save_svmlight(patient_features, hf, 'features_svmlight.train')
    
    events_in, hf_in, feature_map_in = read_csv(VAL_DATA_PATH)
    patient_features, hf = create_features(events_in, hf_in, feature_map_in)
    save_svmlight(patient_features, hf, 'features_svmlight.val')
    
main()

## 3 Predictive Modeling [40 points]

Make sure you have finished section 2 before you start to work on this question because some of the files generated in section 2 (*features_svmlight.train*) will be used in this question.

### 3.1 Model Creation [20 points]

In the previous question, you constructed feature vectors for patients to be used as training data in various predictive models (classifiers). Now you will use this training data (*features_svmlight.train*) in 3 predictive models. 

**Step - a. Implement Logistic Regression, SVM and Decision Tree. Skeleton code is provided in the following code cell.**

In [59]:
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *

import utils


# PLEASE USE THE GIVEN FUNCTION NAME, DO NOT CHANGE IT.
# USE THIS RANDOM STATE FOR ALL OF THE PREDICTIVE MODELS.
# OR THE TESTS WILL NEVER PASS.
RANDOM_STATE = 545510477


#input: X_train, Y_train
#output: Y_pred
def logistic_regression_pred(X_train, Y_train):
    
    """
    TODO: Train a logistic regression classifier using X_train and Y_train.
    Use this to predict labels of X_train. Use default params for the classifier.
    """
    
    # your code here
    clf = LogisticRegression(random_state=RANDOM_STATE).fit(X_train, Y_train)
    return clf.predict(X_train)

X_train, Y_train = utils.get_data_from_svmlight("features_svmlight.train")
print(X_train.shape)
print(logistic_regression_pred(X_train, Y_train))
#display_metrics("Logistic Regression", logistic_regression_pred(X_train, Y_train), Y_train)

#input: X_train, Y_train
#output: Y_pred
def svm_pred(X_train, Y_train):
    
    """
    TODO: Train a SVM classifier using X_train and Y_train.
    Use this to predict labels of X_train. Use default params for the classifier
    """
    
    # your code here
    clf = LinearSVC(random_state=RANDOM_STATE).fit(X_train, Y_train)
    return clf.predict(X_train)

print(svm_pred(X_train, Y_train))
#input: X_train, Y_train
#output: Y_pred
def decisionTree_pred(X_train, Y_train):

    """
    TODO: Train a logistic regression classifier using X_train and Y_train.
    Use this to predict labels of X_train. Use max_depth as 5.
    """
    
    # your code here
    clf = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=5)\
        .fit(X_train, Y_train)
    return clf.predict(X_train)

preds = decisionTree_pred(X_train, Y_train)

accuracy = Y_train == preds
print(accuracy.shape)
print(accuracy.mean())
print(accuracy_score(Y_train, preds))
print(precision_score(Y_train, preds))
#input: Y_pred,Y_true
#output: accuracy, precision, recall, f1-score
def classification_metrics(Y_pred, Y_true):
    
    """
    TODO: Calculate the above mentioned metrics.
    NOTE: It is important to provide the output in the same order.
    """
    
    # your code here
    accuracy = accuracy_score(Y_true, Y_pred)
    precision = precision_score(Y_true, Y_pred)
    recall = recall_score(Y_true, Y_pred)
    f1 = f1_score(Y_true, Y_pred)

    return accuracy, precision, recall, f1

    
#input: Name of classifier, predicted labels, actual labels
def display_metrics(classifierName, Y_pred, Y_true):
    print("______________________________________________")
    print(("Classifier: "+classifierName))
    acc, precision, recall, f1score = classification_metrics(Y_pred,Y_true)
    print(("Accuracy: "+str(acc)))
    print(("Precision: "+str(precision)))
    print(("Recall: "+str(recall)))
    print(("F1-score: "+str(f1score)))
    print("______________________________________________")
    print("")

    
def main():
    X_train, Y_train = utils.get_data_from_svmlight("features_svmlight.train")

    display_metrics("Logistic Regression", logistic_regression_pred(X_train, Y_train), Y_train)
    display_metrics("SVM",svm_pred(X_train, Y_train),Y_train)
    display_metrics("Decision Tree", decisionTree_pred(X_train, Y_train), Y_train)

    
main()

(2485, 1473)
[0. 1. 0. ... 0. 1. 1.]
[0. 1. 0. ... 0. 1. 1.]
(2485,)
0.703420523138833
0.703420523138833
0.6657355679702048
______________________________________________
Classifier: Logistic Regression
Accuracy: 0.856338028169014
Precision: 0.8357933579335793
Recall: 0.937888198757764
F1-score: 0.8839024390243903
______________________________________________

______________________________________________
Classifier: SVM
Accuracy: 0.9070422535211268
Precision: 0.896484375
Recall: 0.9503105590062112
F1-score: 0.9226130653266331
______________________________________________

______________________________________________
Classifier: Decision Tree
Accuracy: 0.703420523138833
Precision: 0.6657355679702048
Recall: 0.9868875086266391
F1-score: 0.7951070336391437
______________________________________________



In [ ]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

from utils import get_data_from_svmlight
from numpy.testing import assert_almost_equal

### 3.1a Training Accuracy [3 points]
X_train, Y_train = get_data_from_svmlight("features_svmlight.train")

# test_accuracy_lr
expected = 0.856338028169014
Y_pred = logistic_regression_pred(X_train, Y_train)
actual = classification_metrics(Y_pred, Y_train)[0]
assert_almost_equal(actual, expected, decimal=2, verbose=False, err_msg="test_accuracy_lr failed!")



**Step - b. Evaluate your predictive models on a separate test dataset in *features_svmlight.val* (binary labels are provided in that svmlight file as the first field). Skeleton code is provided in the following code cell.**

In [60]:
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *

import utils


# PLEASE USE THE GIVEN FUNCTION NAME, DO NOT CHANGE IT.
# USE THIS RANDOM STATE FOR ALL OF THE PREDICTIVE MODELS.
# OR THE TESTS WILL NEVER PASS.
RANDOM_STATE = 545510477


#input: X_train, Y_train and X_test
#output: Y_pred
def logistic_regression_pred(X_train, Y_train, X_test):

    """
    TODO: train a logistic regression classifier using X_train and Y_train. 
    Use this to predict labels of X_test use default params for the classifier.
    """
    
    # your code here
    clf = LogisticRegression(random_state=RANDOM_STATE).fit(X_train, Y_train)
    return clf.predict(X_test)
    

#input: X_train, Y_train and X_test
#output: Y_pred
def svm_pred(X_train, Y_train, X_test):
    
    """
    TODO: Train a SVM classifier using X_train and Y_train.
    Use this to predict labels of X_test use default params for the classifier.
    """
    
    # your code here
    clf = LinearSVC(random_state=RANDOM_STATE).fit(X_train, Y_train)
    return clf.predict(X_test)

    
#input: X_train, Y_train and X_test
#output: Y_pred
def decisionTree_pred(X_train, Y_train, X_test):
    
    """
    TODO: Train a logistic regression classifier using X_train and Y_train.
    Use this to predict labels of X_test.
    IMPORTANT: use max_depth as 5. Else your test cases might fail.
    """
    
    # your code here
    clf = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=5)\
        .fit(X_train, Y_train)
    return clf.predict(X_test)


#input: Y_pred,Y_true
#output: accuracy, precision, recall, f1-score
def classification_metrics(Y_pred, Y_true):
    
    """
    TODO: Calculate the above mentioned metrics.
    NOTE: It is important to provide the output in the same order.
    """
    
    # your code here
    accuracy = accuracy_score(Y_true, Y_pred)
    precision = precision_score(Y_true, Y_pred)
    recall = recall_score(Y_true, Y_pred)
    f1 = f1_score(Y_true, Y_pred)

    return accuracy, precision, recall, f1

    
#input: Name of classifier, predicted labels, actual labels
def display_metrics(classifierName, Y_pred, Y_true):
    print("______________________________________________")
    print(("Classifier: "+classifierName))
    acc, precision, recall, f1score = classification_metrics(Y_pred,Y_true)
    print(("Accuracy: "+str(acc)))
    print(("Precision: "+str(precision)))
    print(("Recall: "+str(recall)))
    print(("F1-score: "+str(f1score)))
    print("______________________________________________")
    print("")

    
def main():
    X_train, Y_train = utils.get_data_from_svmlight("features_svmlight.train")
    X_test, Y_test = utils.get_data_from_svmlight(os.path.join("features_svmlight.val"))

    display_metrics("Logistic Regression", logistic_regression_pred(X_train, Y_train, X_test), Y_test)
    display_metrics("SVM", svm_pred(X_train, Y_train, X_test), Y_test)
    display_metrics("Decision Tree", decisionTree_pred(X_train, Y_train, X_test), Y_test)


main()

______________________________________________
Classifier: Logistic Regression
Accuracy: 0.6937086092715232
Precision: 0.7345360824742269
Recall: 0.776566757493188
F1-score: 0.7549668874172186
______________________________________________

______________________________________________
Classifier: SVM
Accuracy: 0.640728476821192
Precision: 0.7038043478260869
Recall: 0.7057220708446866
F1-score: 0.7047619047619047
______________________________________________

______________________________________________
Classifier: Decision Tree
Accuracy: 0.6821192052980133
Precision: 0.6611418047882136
Recall: 0.9782016348773842
F1-score: 0.789010989010989
______________________________________________



In [ ]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

from utils import get_data_from_svmlight
from numpy.testing import assert_almost_equal

### 3.1b Prediction Accuracy [3 points]
X_train, Y_train = get_data_from_svmlight("features_svmlight.train")
X_test, Y_test = get_data_from_svmlight("features_svmlight.val")

# test_accuracy_lr
expected = 0.6937086092715232
Y_pred = logistic_regression_pred(X_train, Y_train, X_test)
actual = classification_metrics(Y_pred, Y_test)[0]
assert_almost_equal(actual, expected, decimal=2, verbose=False, err_msg="test_accuracy_lr failed!")



### 3.2 Model Validation [20 points]

In order to fully utilize the available data and obtain more reliable results, machine learning practitioners use cross-validation to evaluate and improve their predictive models. You will demonstrate using two cross-validation strategies against SVD. 

- K-fold: Divide all the data into $k$ groups of samples. Each time $\frac{1}{k}$ samples will be used as test data and the remaining samples as training data.
- Randomized K-fold: Iteratively random shuffle the whole dataset and use top specific percentage of data as training and the rest as test. 

**Implement the two cross-validation strategies.**
- **K-fold:** Use the number of iterations k=5; 
- **Randomized K-fold**: Use a test data percentage of 20\% and k=5 for the number of iterations for Randomized

In [73]:
from sklearn.model_selection import KFold, ShuffleSplit
from numpy import mean

import utils


# PLEASE USE THE GIVEN FUNCTION NAME, DO NOT CHANGE IT.
# USE THIS RANDOM STATE FOR ALL OF THE PREDICTIVE MODELS.
# OR THE TESTS WILL NEVER PASS.
RANDOM_STATE = 545510477


#input: training data and corresponding labels
#output: accuracy, f1
def get_f1_kfold(X, Y, k=5):
    
    """
    TODO: First get the train indices and test indices for each iteration.
    Then train the classifier accordingly.
    Report the mean f1 score of all the folds.
    
    Note that you do not need to set random_state for KFold, as it has no effect since shuffle is False by default.
    """
    
    # your code here
    kf = KFold(n_splits=k)
    f1t = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        clf = LinearSVC(random_state=RANDOM_STATE).fit(X_train, Y_train)
        Y_pred = clf.predict(X_test)
        f1t = f1t + f1_score(Y_test, Y_pred)

    return f1t / k

X,Y = utils.get_data_from_svmlight("features_svmlight.train")
print(X.shape)
print(get_f1_kfold(X, Y))
    
#input: training data and corresponding labels
#output: accuracy, f1
def get_f1_randomisedCV(X, Y, iterNo=5, test_percent=0.20):

    """
    TODO: First get the train indices and test indices for each iteration.
    Then train the classifier accordingly.
    Report the mean f1 score of all the iterations.
    
    Note that you need to set random_state for ShuffleSplit
    """

    # your code here
    ss = ShuffleSplit(n_splits=iterNo, random_state=RANDOM_STATE, test_size=test_percent)
    f1t = 0
    for train_index, test_index in ss.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        clf = LinearSVC(random_state=RANDOM_STATE).fit(X_train, Y_train)
        Y_pred = clf.predict(X_test)
        f1t = f1t + f1_score(Y_test, Y_pred)

    return f1t / iterNo

    
def main():
    X,Y = utils.get_data_from_svmlight("features_svmlight.train")
    print("Classifier: SVD")
    f1_k = get_f1_kfold(X,Y)
    print(("Average F1 Score in KFold CV: "+str(f1_k)))
    f1_r = get_f1_randomisedCV(X,Y)
    print(("Average F1 Score in Randomised CV: "+str(f1_r)))


main()

(2485, 1473)
0.7258461959533061
Classifier: SVD
Average F1 Score in KFold CV: 0.7258461959533061
1988 497
1988 497
1988 497
1988 497
1988 497
Average F1 Score in Randomised CV: 0.7195678940019832


In [ ]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

from numpy.testing import assert_almost_equal

### 3.2 Cross Validation F1 [10 points]
# test_f1_cv_kfold
expected = 0.7258461959533061
X, Y = get_data_from_svmlight("features_svmlight.train")
actual = get_f1_kfold(X, Y)
assert_almost_equal(actual, expected, decimal=2, verbose=False, err_msg="test_f1_cv_kfold failed!")

